In [21]:
from pymongo import MongoClient
import googlemaps
from datetime import datetime
import os
from dotenv import load_dotenv
load_dotenv()

gmaps =googlemaps.Client(key=os.getenv("KEY"))

In [2]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [3]:
db, coll = connectCollection('companies','companies')

In [4]:
companies = list(coll.find({"offices.city": "San Jose","founded_year": {"$lt": 2008},"offices.longitude":{"$ne":None},"offices.latitude":{"$ne": None}}))

In [5]:
print(len(companies))

89


In [6]:
print(companies[0]['offices'][0]['longitude'], companies[0]['offices'][0]['latitude'])

-121.930035 37.295005


In [7]:
set(companies[0]['offices'][0])

{'address1',
 'address2',
 'city',
 'country_code',
 'description',
 'latitude',
 'longitude',
 'state_code',
 'zip_code'}

In [8]:
def getLocation(company):
    location=[]
    for i in range(len(company['offices'])):        
        longitude = company['offices'][i]['longitude']
        latitude = company['offices'][i]['latitude']
        loc = {
            'type':'Point',
            'coordinates':[longitude,latitude]
        }
        location.append(loc)
    return location

In [9]:
for company in companies:
    value = {"$set": {'location':getLocation(company)}}
    coll.update_one(company,value)

In [10]:
for company in range(len(companies)):
    print(companies[company]['location'][0]['coordinates'])

[-121.930035, 37.295005]
[-121.95377, 37.408802]
[-121.979453, 37.321798]
[-121.877636, 37.320052]
[-121.927696, 37.294465]
[-121.948399, 37.32052]
[-122.006539, 37.332271]
[-95.712891, 37.09024]
[-121.939993, 37.410651]
[-121.9477639, 37.4050988]
[-121.789862, 37.264749]
[-121.9053, 37.244187]
[-121.92902, 37.395923]
[-121.884399, 37.332828]
[-122.031476, 37.308591]
[-121.979453, 37.321798]
[-121.971487, 37.381034]
[-122.0879281, 37.3973787]
[-121.915949, 37.367293]
[-121.89435, 37.337756]
[-121.957018, 37.421731]
[-121.873881, 37.316466]
[-122.087471, 37.419041]
[-121.873881, 37.316466]
[-121.873881, 37.316466]
[-121.89053, 37.332948]
[-121.920558, 37.371056]
[-121.771853, 37.324803]
[-121.860571, 37.285978]
[-121.957018, 37.421729]
[-121.9219538, 37.3706126]
[-122.4010019, 37.7924253]
[-122.031435, 37.299332]
[-121.9219538, 37.3706126]
[-122.4010019, 37.7924253]
[-122.031435, 37.299332]
[-121.9486567, 37.4155698]
[-122.363276, 47.622263]
[-122.012808, 37.383526]
[-122.0497918, 37.39

In [11]:
companies[0]

{'_id': ObjectId('52cdef7c4bab8bd675297d9b'),
 'name': 'eBay',
 'permalink': 'ebay',
 'crunchbase_url': 'http://www.crunchbase.com/company/ebay',
 'homepage_url': 'http://ebay.com',
 'blog_url': '',
 'blog_feed_url': '',
 'twitter_username': 'eBay',
 'category_code': 'web',
 'number_of_employees': 15000,
 'founded_year': 1995,
 'founded_month': 9,
 'founded_day': 1,
 'deadpooled_year': None,
 'deadpooled_month': None,
 'deadpooled_day': None,
 'deadpooled_url': None,
 'tag_list': 'auction, bid, search, market',
 'alias_list': '',
 'email_address': '',
 'phone_number': '',
 'description': 'Online Marketplace',
 'created_at': 'Wed Jun 06 13:03:13 UTC 2007',
 'updated_at': 'Tue Sep 10 03:38:03 UTC 2013',
 'overview': '<p>Founded in 1995 in San Jose, CA, eBay connects millions of buyers and sellers globally in the world&#8217;s largest online marketplace, utilizing <a href="/company/paypal" title="PayPal" rel="nofollow">PayPal</a> to ensure secure transactions. The company also operates sp

In [12]:
import folium

In [26]:
starbucks_list = gmaps.places('starbucks,san jose,ca')

In [41]:
map_city=folium.Map(location=[37.3382,-121.8863], zoom_start=12)
for company in companies:
    folium.Marker(company['location'][0]['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='gray'), 
                   ).add_to(map_city)
for starbuck in range(len(starbucks_list)):
    folium.Marker([starbucks_list['results'][starbuck]['geometry']['location'],
                  radius=2, 
                  icon=folium.Icon(icon='cloud',color='green'),
                 ).add_to(map_city)

ValueError: Expected two (lat, lon) values for location, instead got: [{'lat': 37.387198, 'lng': -121.861847}].

In [42]:
# Gray: Empresas con mas de diez años.
# Green : Starbucks.
map_city